<a href="https://colab.research.google.com/github/debbiedebdeb/MDG/blob/master/data_UTMK_%26_cnt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 좌표변환

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pyproj #pyproj 설치 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 5.1 MB/s 


In [4]:
import pandas as pd
import numpy as np
import pyproj
import folium
from pyproj import Transformer

## WGS84 to UTMK

### 도서관 위경도 변경

In [5]:
lb = pd.read_excel('/content/서울시 공공도서관 현황정보.xlsx')
lb.head(1)

,도서관 일련번호,도서관명,구 코드,구명,주소,전화번호,홈페이지 URL,운영시간,정기 휴관일,도서관 구분,도서관 구분명,위도,경도
0,1935,강남구립못골도서관,8,강남구,서울특별시 강남구 자곡로 116,02-459-5522,http://library.gangnam.go.kr/mglib,평일 : 09:00~22:00 주말 : 09:00~22:00,매주 화요일 및 법정공휴일,1,공공도서관,37.471538,127.096582


### 위 방법은 더이상 이용되지 않는 방법. 

In [6]:
 #Projection 정의 
# proj_utmk = Proj(init = 'epsg:5178')  # UTM-K(Bassel) 도로명주소 지도 사용
# proj_WGS84 = Proj(init='epsg:4326') # Wgs84 경도/위도, GPS사용 전지구 좌표

NameError: ignored

In [ ]:
# 변경할 col 
# lb_la = lb['위도']
# lb_lo = lb['경도']

In [ ]:
#도서관 좌표 UTMK로 변환 함수

# def transform_to_utmk(df):
#     df['trans_la'], df['trans_lo'] = pyproj.transform(proj_WGS84, proj_utmk, df['위도'].tolist(), df['경도'].tolist())
#     return df.head(5)

In [ ]:
# test..실패 
# pyproj.transform(proj_WGS84, proj_utmk, 37.553230, 126.972551)

### 업데이트된 변환 방법 

pyproj 문서에 따르면 

"pyproj.transformer.transform() and pyproj.transformer.itransform() are deprecated." 이라고 적혀있다. 
 * pyproj.transformer.transform() and pyproj.transformer.itransform()은 더이상 이용이 불가하다는 뜻.
 * 다른방안으로 pyproj 2 style을 이용했습니다. 

https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrade-transformer

In [7]:
from pyproj import Transformer

In [8]:
# transformer 정의 
transformer = Transformer.from_crs("epsg:4326", "epsg:5178")

In [9]:
# 서울역 좌표로 test
transformer.transform(37.553230, 126.972551) #성공

(1950256.3879044417, 953600.9747718363)

In [10]:
lb['trans_la'], lb['trans_lo'] = transformer.transform(lb['위도'], lb['경도'])
lb.head(1)

,도서관 일련번호,도서관명,구 코드,구명,주소,전화번호,홈페이지 URL,운영시간,정기 휴관일,도서관 구분,도서관 구분명,위도,경도,trans_la,trans_lo
0,1935,강남구립못골도서관,8,강남구,서울특별시 강남구 자곡로 116,02-459-5522,http://library.gangnam.go.kr/mglib,평일 : 09:00~22:00 주말 : 09:00~22:00,매주 화요일 및 법정공휴일,1,공공도서관,37.471538,127.096582,1.941139e+06,964517.755038


In [11]:
# 도서관 데이터에서 x, y 컬럼만 빼서 데이터 생성 
lb_xy = lb[['trans_la', 'trans_lo']]
lb_xy.head(1)

,trans_la,trans_lo
0,1.941139e+06,964517.755038


### 함수만들기 

In [12]:
# 성공했으니 토대로 함수를 만들어 모든 시설의 좌표를 변환해보자 

def to_utmk(df, x, y) :
    df['utmk_la'], df['utmk_lo'] = transformer.transform(df[x], df[y])

    return df.head(1)


In [13]:
# 함수 test
to_utmk(lb, '위도', '경도')

,도서관 일련번호,도서관명,구 코드,구명,주소,전화번호,홈페이지 URL,운영시간,정기 휴관일,도서관 구분,도서관 구분명,위도,경도,trans_la,trans_lo,utmk_la,utmk_lo
0,1935,강남구립못골도서관,8,강남구,서울특별시 강남구 자곡로 116,02-459-5522,http://library.gangnam.go.kr/mglib,평일 : 09:00~22:00 주말 : 09:00~22:00,매주 화요일 및 법정공휴일,1,공공도서관,37.471538,127.096582,1.941139e+06,964517.755038,1.941139e+06,964517.755038


In [14]:
lb_xy = lb[['utmk_la', 'utmk_lo']]
lb_xy

,utmk_la,utmk_lo
0,1.941139e+06,964517.755038
1,1.943934e+06,958823.237202
2,1.946238e+06,959290.709073
3,1.944175e+06,961915.565734
4,1.943023e+06,959424.568175
...,...,...
198,1.955684e+06,965228.177283
199,1.953992e+06,963775.627689
200,1.957078e+06,963731.956580
201,1.954632e+06,962997.759628


### 버스정류장 데이터 변환

In [15]:
# 버스정류장 데이터 

busstop = pd.read_excel('/content/서울시버스정류소좌표데이터(2022.08.24).xlsx')
busstop.head(1)

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y
0,100000001,1001,종로2가사거리,126.987786,37.569764


In [16]:
#busstop 좌표 X = 경도 , Y = 위도 
to_utmk(busstop, '좌표Y', '좌표X')

,NODE_ID,ARS-ID,정류소명,좌표X,좌표Y,utmk_la,utmk_lo
0,100000001,1001,종로2가사거리,126.987786,37.569764,1.952083e+06,954956.649795


In [17]:
busstop_xy = busstop[['utmk_la', 'utmk_lo']]
busstop_xy.columns = ['bus_utmk_la', 'bus_utmk_lo']

In [18]:
busstop_xy 

,bus_utmk_la,bus_utmk_lo
0,1.952083e+06,954956.649795
1,1.953124e+06,955733.526113
2,1.953515e+06,955891.899964
3,1.951952e+06,954931.302542
4,1.953904e+06,956194.535997
...,...,...
11162,1.949878e+06,968330.786354
11163,1.949897e+06,968394.984755
11164,1.948024e+06,966923.417967
11165,1.948302e+06,967095.297212


### 시니어 배움터 시설목록 좌표전환

In [19]:
bwt = pd.read_csv('/content/시니어 배움터 시설 목록 2021.csv')
bwt.head(1)

,ESNTL_ID,LCLAS_NM,MLSFC_NM,FCLTY_NM,CTPRVN_CD,CTPRVN_NM,SIGNGU_CD,SIGNGU_NM,LEGALDONG_CD,LEGALDONG_NM,...,BULD_MANAGE_CD,TEL_NO,ZIP_NO,HMPG_URL,FCLTY_LA,FCLTY_LO,ORIGIN_NM,ADIT_DC,UPDT_DT,REGIST_DT
0,KCOEFPO21N000000001,문화시설,문화시설_복지_시니어,종로노인종합복지관,11.0,서울특별시,11110.0,종로구,1.111041e+11,이화동,...,NaN,27429500,3100.0,NaN,37.576762,127.006013,문화정보원,NaN,20220411120000,20220411120000


In [20]:
bwt = bwt[['FCLTY_LA', 'FCLTY_LO']]
bwt.head()

,FCLTY_LA,FCLTY_LO
0,37.576762,127.006013
1,37.575907,126.958126
2,37.576013,126.985804
3,37.552461,127.009313
4,37.556540,127.014318


In [21]:
#변환하기 
to_utmk(bwt,'FCLTY_LA', 'FCLTY_LO' )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,FCLTY_LA,FCLTY_LO,utmk_la,utmk_lo
0,37.576762,127.006013,1.952851e+06,956570.321273


In [22]:
bwt_xy = bwt[['utmk_la','utmk_lo']]
bwt_xy.columns = ['bwt_utmk_la', 'bwt_utmk_lo']

In [23]:
bwt_xy.head(1)

,bwt_utmk_la,bwt_utmk_lo
0,1.952851e+06,956570.321273


### 공원 좌표전환 

In [24]:
park = pd.read_csv('/content/시니어 나들이 추천 목록 2021.csv')
park.head(1)

,ESSN_ID,LCLAS_NM,MLSFC_NM,FCLTY_NM,CTPR_CD,CTPR_NM,SIGNGU_CD,SIGNGU_NM,LEGALDONG_CD,LEGALDONG_NM,...,BULD_MANAGE_CD,TELNO,ZIP,HMPG_URL,FCLTY_LA,FCLTY_LO,OGN_NM,ADIT_DC,UPDATE_DT,RGSTN_DT
0,KCOWRPO21N000000513,자연,자연_공원,방 죽,NaN,서울특별시,NaN,중랑구,NaN,망우동,...,NaN,0220942364,NaN,NaN,37.60238,127.1027,문화정보원,NaN,20220411120000,20220411120000


In [25]:
park.columns

Index(['ESSN_ID', 'LCLAS_NM', 'MLSFC_NM', 'FCLTY_NM', 'CTPR_CD', 'CTPR_NM',
       'SIGNGU_CD', 'SIGNGU_NM', 'LEGALDONG_CD', 'LEGALDONG_NM', 'RN_CD',
       'FCLTY_ROAD_NM_ADDR', 'LNNO_ADRES', 'ADDR_ENG_NM', 'ADSTRD_CD',
       'BULD_NM', 'BULD_MANAGE_CD', 'TELNO', 'ZIP', 'HMPG_URL', 'FCLTY_LA',
       'FCLTY_LO', 'OGN_NM', 'ADIT_DC', 'UPDATE_DT', 'RGSTN_DT'],
      dtype='object')

In [27]:
to_utmk(park, 'FCLTY_LA','FCLTY_LO', )

,ESSN_ID,LCLAS_NM,MLSFC_NM,FCLTY_NM,CTPR_CD,CTPR_NM,SIGNGU_CD,SIGNGU_NM,LEGALDONG_CD,LEGALDONG_NM,...,ZIP,HMPG_URL,FCLTY_LA,FCLTY_LO,OGN_NM,ADIT_DC,UPDATE_DT,RGSTN_DT,utmk_la,utmk_lo
0,KCOWRPO21N000000513,자연,자연_공원,방 죽,NaN,서울특별시,NaN,중랑구,NaN,망우동,...,NaN,NaN,37.60238,127.1027,문화정보원,NaN,20220411120000,20220411120000,1.955653e+06,965119.852063


In [28]:
park_xy = park[['utmk_la','utmk_lo']]
park_xy.columns = ['park_utmk_la', 'park_utmk_lo']

## 주소 to WGS84 to UTMK

In [ ]:
from pyproj import Transformer
transformer = Transformer.from_crs("epsg:4326", "epsg:4326")
transformer.transform(12, 12)

### 병원 변환

In [39]:
hospital = pd.read_excel('/content/drive/MyDrive/공유폴더/위경도UTMK변환/병원 주소.xlsx')

In [41]:
address = hospital['지번주소']
address[0]

'서울특별시 양천구 신정동 899-1'

In [54]:
# 주소에서 좌표 뽑아내기 

from geopy.geocoders import Nominatim 

geolocoder = Nominatim(user_agent = 'South Korea', timeout = None)

def geocoding(address) : 
    try : 
        geo= geolocoder.geocode(address)
        crd = (str(geo.latitude), str(geo.longitude))
        return crd
    
    except : 
        return (0,0)

In [55]:
lats = []
logs = []

for address in hospital['지번주소'] : 
    crd = geocoding(address)
    lats.append(crd[0])
    logs.append(crd[1])

lats = pd.Series(lats)
logs = pd.Series(logs)

In [56]:
hospital = pd.concat([hospital, lats], axis = 1)
hospital = pd.concat([hospital, logs], axis = 1)

In [61]:
hospital.columns

Index(['Unnamed: 0',       '지번주소',       '사업장명',    '좌표정보(X)',    '좌표정보(Y)',
              '자치구',          '동', 'Unnamed: 7',            0,            0],
      dtype='object')

In [63]:
hospital.drop(columns = 'Unnamed: 0', inplace = True)
hospital.columns 

Index(['지번주소', '사업장명', '좌표정보(X)', '좌표정보(Y)', '자치구', '동', 'Unnamed: 7', 0, 0], dtype='object')

In [64]:
hospital.columns = ['지번주소', '사업장명', '좌표정보(X)', '좌표정보(Y)', '자치구', '동', 'Unnamed: 7', '위도', '경도']

In [65]:
hospital_xy = hospital[['위도', '경도']]

In [ ]:
hospital_xy
# (0,0) 즉, 위경도 변환이 안된 주소들은 직접 입력을 해주거나 해야한다. 
# 하지만 본 코딩은 정리 코딩이라 해당 작업을 skip

# 주민센터 및 복지관 기준으로 편의시설 개수 찾기 

* 유클리드 계산을 이용 

In [71]:
#주민센터 위경도가 있는 파일 불러오기 
df = pd.read_excel('/content/drive/MyDrive/공유폴더/완성데이터/주변특성_개수_주소, 위도, 경도 포함.xlsx')
df = df.iloc[:, :6]
df.head()

,name,address,X,Y,위도UTMK,경도UTMK
0,청운효자동 주민센터,서울특별시 종로구 자하문로 92,37.584083,126.970636,1.953680e+06,953451.030567
1,사직동 주민센터,서울특별시 종로구 사직로9길 1,37.576191,126.968829,1.952806e+06,953286.588072
2,종로1-4가동 주민센터,서울특별시 종로구 종로17길 8,37.570728,126.988912,1.952190e+06,955056.678912
3,삼청동 주민센터,서울특별시 종로구 삼청로 107,37.584979,126.981782,1.953774e+06,954435.679506
4,가회동 주민센터,서울특별시 종로구 북촌로 35,37.580025,126.984805,1.953223e+06,954699.580385


In [72]:
df['버스정류장(100m)'] = 0

In [78]:
x = df['위도UTMK']
y = df['경도UTMK']
x1 = busstop_xy['bus_utmk_la']
y1 = busstop_xy['bus_utmk_lo']

In [83]:
#유클리드 거리 공식

for i in range(len(df)) :
    for j in range(len(busstop_xy)) :
        value = False 
        value = np.sqrt( (x[i] - x1[j])**2 + (y[i]-y1[j])**2) <=200

        if value == True : 
            df['버스정류장(100m)'][i] += int(value)
        else :
            pass
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,name,address,X,Y,위도UTMK,경도UTMK,버스정류장(100m)
0,청운효자동 주민센터,서울특별시 종로구 자하문로 92,37.584083,126.970636,1.953680e+06,953451.030567,4
1,사직동 주민센터,서울특별시 종로구 사직로9길 1,37.576191,126.968829,1.952806e+06,953286.588072,5
2,종로1-4가동 주민센터,서울특별시 종로구 종로17길 8,37.570728,126.988912,1.952190e+06,955056.678912,8
3,삼청동 주민센터,서울특별시 종로구 삼청로 107,37.584979,126.981782,1.953774e+06,954435.679506,4
4,가회동 주민센터,서울특별시 종로구 북촌로 35,37.580025,126.984805,1.953223e+06,954699.580385,7
...,...,...,...,...,...,...,...
444,관악구,서울시 관악구 관악로145,37.478240,126.951502,1.941947e+06,951693.357923,5
445,서초구,서울시 서초구 남부순환로2584,37.483725,127.032566,1.942517e+06,958863.832385,5
446,강남구,서울시 강남구 학동로 426,37.517553,127.047466,1.946264e+06,960199.141484,2
447,송파구,서울시 송파구 올림픽로 326,37.514596,127.106067,1.945913e+06,965376.385238,2


In [84]:
df['버스정류장(100m)'].value_counts()

4     75
2     70
3     57
5     52
6     45
0     42
1     30
7     23
8     20
10    13
9     12
11     3
12     3
15     2
13     2
Name: 버스정류장(100m), dtype: int64